In [5]:
import json

with open("/home/dgmastertemple/Documents/GitHub/create-audio-bible/projects/Ephesians - Tim Conway/json/readings.json", "r") as f:
    data: dict = json.load(f)

In [10]:
from dataclasses import dataclass
@dataclass
class Reading:
    id: str
    start_time: float
    end_time: float
    start_seg: int
    end_seg: int


In [12]:

for ref, finds in data.items():
    print(ref)
    r = Reading(**finds[0])
    print(r)
    break

Ephesians 1:1
Reading(id='69161619300', start_time=35.624, end_time=47.333, start_seg=14, end_seg=37)


In [1]:
from mutagen.id3 import ID3, APIC

input_file = "/home/dgmastertemple/Documents/GitHub/create-audio-bible/projects/Ephesians - Tim Conway/export/verses/Ephesians 1:1 - Tim Conway.mp3"
cover_image = "/home/dgmastertemple/Documents/GitHub/create-audio-bible/sources/ill_be_honest.jpeg"

def add_album_art(input_file: str, cover_image: str):
    audio = ID3(input_file)
    with open(cover_image, 'rb') as image_file:
        image_data = image_file.read()
    mime='image/jpeg'
    if cover_image.endswith(".png"):
        mime='image/png'
    album_art = APIC(
        encoding=3,  # UTF-8
        mime=mime,
        type=3,  # Cover (front)
        desc='Cover (Front)',
        data=image_data
    )
    audio['APIC'] = album_art
    audio.save(input_file, v2_version=3)

add_album_art(input_file, cover_image)